 # Question 4, taking the template of question 3 i guess

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Flatten, Input, Dense, SimpleRNN, GRU, LSTM, Dropout, Activation, BatchNormalization, LayerNormalization, Dropout
from keras import initializers, regularizers, optimizers, backend

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Activation, BatchNormalization, Dropout
from keras import initializers, regularizers, optimizers, backend




# ANN

In [3]:
# cleaning out the NaNs
# this will be loading in the regression dataset
# Replace 'your_file.csv' with the path to your CSV file
file_path = 'data/Air_pollution.csv'

# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(file_path)

time_offset_list = [24, 72, 168]
mse_train_list = []
mse_val_list = []
mse_test_list = []

for time_offset in time_offset_list:
    # taking out the nessesary data
    new_df = df[['month', 'day', 'hour', 'pm2.5', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']]

    # dropping out the NaN numbers i guess
    new_df = new_df.dropna()

    # data preprocessing
    # using default values + month, since that was what worked best i guess
    asset_input = pd.DataFrame(new_df, columns=['month', 'DEWP', 'TEMP', 'PRES', 'Iws'])
    x_all = asset_input.values

    asset_output = pd.DataFrame(new_df, columns=['pm2.5'])
    y_all = asset_output.values

    # finding out the length of the dataset, and then getting the first 90% as the data i guess
    splits = int(0.9 * x_all.shape[0])

    timestep = 50

    time_offset = time_offset

    # in this case, i have to offset the training data by 24 hours, 72, and 168 hours. I will be doing this by adding empty zero rows into the data, and then trimming data off the end so that no invalid data is obtained i guess
    # bumping the x data forward by 24 instead of pushing y back by 24 i guess
    print(x_all.shape, y_all.shape)
    zeros_to_append = np.zeros((time_offset, 5))
    x_all = np.concatenate((zeros_to_append, x_all))
    zeros_to_append = np.zeros((time_offset, 1))
    y_all = np.concatenate((y_all, zeros_to_append))
    print(x_all.shape, y_all.shape)

    # slicing out the useless zero data at the end and at the start. The array should now be smaller, but should be offsetted the right amount
    # idk how the nan values are gonna affect this. Hopefully it does not affect it too much
    x_all = x_all[time_offset:-time_offset, :]
    y_all = y_all[time_offset:-time_offset, :]
    print(x_all.shape, y_all.shape)

    x_real = x_all[0:splits, :]
    y_real = y_all[0:splits, :]

    # testing samples
    x_test_real = x_all[splits-timestep:, :]
    y_test_real = y_all[splits-timestep:, :]

    # scaling the samples accordingly
    # most of them fall within a range, and has min and max value, and as such, normalization is choosen i guess
    # Normalization
    scaler_x = MinMaxScaler()
    scaler_x.fit(x_real)
    x_scale = scaler_x.transform(x_real)

    scaler_y = MinMaxScaler()
    scaler_y.fit(y_real)
    y_scale = scaler_y.transform(y_real)

    # setting up the training and validation dataset
    data_generator = TimeseriesGenerator(x_scale, y_scale, length=timestep, sampling_rate=1, batch_size=10000)
    x_timestep, y_timestep = data_generator[0]

    x_train, x_val, y_train, y_val = train_test_split(x_timestep, y_timestep, test_size=0.1, random_state=7567)


    # generating the testing data as well i guess
    x_test_scale = scaler_x.transform(x_test_real)
    y_test_scale = scaler_y.transform(y_test_real)

    data_generator = TimeseriesGenerator(x_test_scale, y_test_scale, length=timestep, sampling_rate=1, batch_size=10000)
    x_test, y_test = data_generator[0]




    # network section starts here
    backend.clear_session()
    init_weight = initializers.GlorotUniform(seed=7567)
    regu_weight = regularizers.l2(0.01)

    model = Sequential()
    model.add(Flatten(input_shape=(timestep,5)))
    model.add(Dense(100, kernel_initializer=init_weight, kernel_regularizer=regu_weight))
    model.add(BatchNormalization())
    model.add(Activation('ReLU'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer=init_weight, kernel_regularizer=regu_weight))

    my_optmz = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=my_optmz, loss='mean_squared_error', metrics='mse')
    model_hist = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test,y_test), verbose=0)

    # predict the train and validation as well i guess


    y_test_pred_scale = model.predict(x_test, verbose=0)
    mse_test_scale = mean_squared_error(y_test, y_test_pred_scale)

    y_test_pred_real = scaler_y.inverse_transform(y_test_pred_scale)
    mse_test_real = mean_squared_error(y_test_real[timestep:,0], y_test_pred_real)


    mse_train_list.append(model_hist.history['mse'][-1])
    mse_val_list.append(model_hist.history['val_mse'][-1])
    # grabbing the scaled mse
    mse_test_list.append(mse_test_scale)

    print(f'timestep of {timestep}')
    print("Mean squared error of the scaled testing dataset:", mse_test_scale)
    print("Mean squared error of the real testing dataset (USD):", mse_test_real)

print(mse_train_list)
print(mse_val_list)
print(mse_test_list)

(41757, 5) (41757, 1)
(41781, 5) (41781, 1)
(41733, 5) (41733, 1)
timestep of 50
Mean squared error of the scaled testing dataset: 0.01313077854279026
Mean squared error of the real testing dataset (USD): 12973.681923665597
(41757, 5) (41757, 1)
(41829, 5) (41829, 1)
(41685, 5) (41685, 1)
timestep of 50
Mean squared error of the scaled testing dataset: 0.02009933836980414
Mean squared error of the real testing dataset (USD): 19858.86988439936
(41757, 5) (41757, 1)
(41925, 5) (41925, 1)
(41589, 5) (41589, 1)


KeyboardInterrupt: 

 # RNN

In [22]:
# cleaning out the NaNs
# this will be loading in the regression dataset
# Replace 'your_file.csv' with the path to your CSV file
file_path = 'data/Air_pollution.csv'

# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(file_path)

time_offset_list = [24, 72, 168]
mse_train_list = []
mse_val_list = []
mse_test_list = []

for time_offset in time_offset_list:
    # taking out the nessesary data
    new_df = df[['month', 'day', 'hour', 'pm2.5', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']]

    # dropping out the NaN numbers i guess
    new_df = new_df.dropna()

    # data preprocessing
    # using default values + month, since that was what worked best i guess
    asset_input = pd.DataFrame(new_df, columns=['month', 'DEWP', 'TEMP', 'PRES', 'Iws'])
    x_all = asset_input.values

    asset_output = pd.DataFrame(new_df, columns=['pm2.5'])
    y_all = asset_output.values

    # finding out the length of the dataset, and then getting the first 90% as the data i guess
    splits = int(0.9 * x_all.shape[0])

    timestep = 50

    time_offset = time_offset

    # in this case, i have to offset the training data by 24 hours, 72, and 168 hours. I will be doing this by adding empty zero rows into the data, and then trimming data off the end so that no invalid data is obtained i guess
    # bumping the x data forward by 24 instead of pushing y back by 24 i guess
    print(x_all.shape, y_all.shape)
    zeros_to_append = np.zeros((time_offset, 5))
    x_all = np.concatenate((zeros_to_append, x_all))
    zeros_to_append = np.zeros((time_offset, 1))
    y_all = np.concatenate((y_all, zeros_to_append))
    print(x_all.shape, y_all.shape)

    # slicing out the useless zero data at the end and at the start. The array should now be smaller, but should be offsetted the right amount
    # idk how the nan values are gonna affect this. Hopefully it does not affect it too much
    x_all = x_all[time_offset:-time_offset, :]
    y_all = y_all[time_offset:-time_offset, :]
    print(x_all.shape, y_all.shape)

    x_real = x_all[0:splits, :]
    y_real = y_all[0:splits, :]

    # testing samples
    x_test_real = x_all[splits-timestep:, :]
    y_test_real = y_all[splits-timestep:, :]

    # scaling the samples accordingly
    # most of them fall within a range, and has min and max value, and as such, normalization is choosen i guess
    # Normalization
    scaler_x = MinMaxScaler()
    scaler_x.fit(x_real)
    x_scale = scaler_x.transform(x_real)

    scaler_y = MinMaxScaler()
    scaler_y.fit(y_real)
    y_scale = scaler_y.transform(y_real)

    # setting up the training and validation dataset
    data_generator = TimeseriesGenerator(x_scale, y_scale, length=timestep, sampling_rate=1, batch_size=10000)
    x_timestep, y_timestep = data_generator[0]

    x_train, x_val, y_train, y_val = train_test_split(x_timestep, y_timestep, test_size=0.1, random_state=7567)


    # generating the testing data as well i guess
    x_test_scale = scaler_x.transform(x_test_real)
    y_test_scale = scaler_y.transform(y_test_real)

    data_generator = TimeseriesGenerator(x_test_scale, y_test_scale, length=timestep, sampling_rate=1, batch_size=10000)
    x_test, y_test = data_generator[0]




    # network section starts here
    backend.clear_session()
    init_weight = initializers.GlorotUniform(seed=7567)
    regu_weight = regularizers.l2(0.001)

    model = Sequential()
    model.add(Input(shape=(timestep,5)))
    model.add(SimpleRNN(100, activation='tanh', return_sequences=False, kernel_initializer=init_weight, kernel_regularizer=regu_weight, recurrent_dropout=0))
    #model.add(LayerNormalization()) #optional, default activation must be changed to None
    #model.add(Activation('tanh')) #separately apply activation function if using layer normalization
    model.add(Dense(1, kernel_initializer=init_weight, kernel_regularizer=regu_weight))

    my_optmz = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=my_optmz, loss='mean_squared_error', metrics='mse')
    model_hist = model.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_val,y_val), verbose=0)




    # predict the train and validation as well i guess
    y_test_pred_scale = model.predict(x_test, verbose=0)
    mse_test_scale = mean_squared_error(y_test, y_test_pred_scale)

    y_test_pred_real = scaler_y.inverse_transform(y_test_pred_scale)
    mse_test_real = mean_squared_error(y_test_real[timestep:,0], y_test_pred_real)


    mse_train_list.append(model_hist.history['mse'][-1])
    mse_val_list.append(model_hist.history['val_mse'][-1])
    # grabbing the scaled mse
    mse_test_list.append(mse_test_scale)

    print(f'timestep of {timestep}')
    print("Mean squared error of the scaled testing dataset:", mse_test_scale)
    print("Mean squared error of the real testing dataset (USD):", mse_test_real)

print(mse_train_list)
print(mse_val_list)
print(mse_test_list)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 100)               5100      
                                                                 
 batch_normalization (Batch  (None, 100)               400       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                        

# Gated recurrent neural network

In [ ]:
# cleaning out the NaNs
# this will be loading in the regression dataset
# Replace 'your_file.csv' with the path to your CSV file
file_path = 'data/Air_pollution.csv'

# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(file_path)

time_offset_list = [24, 72, 168]
mse_train_list = []
mse_val_list = []
mse_test_list = []

for time_offset in time_offset_list:
    # taking out the nessesary data
    new_df = df[['month', 'day', 'hour', 'pm2.5', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']]

    # dropping out the NaN numbers i guess
    new_df = new_df.dropna()

    # data preprocessing
    # using default values + month, since that was what worked best i guess
    asset_input = pd.DataFrame(new_df, columns=['month', 'DEWP', 'TEMP', 'PRES', 'Iws'])
    x_all = asset_input.values

    asset_output = pd.DataFrame(new_df, columns=['pm2.5'])
    y_all = asset_output.values

    # finding out the length of the dataset, and then getting the first 90% as the data i guess
    splits = int(0.9 * x_all.shape[0])

    timestep = 50

    time_offset = time_offset

    # in this case, i have to offset the training data by 24 hours, 72, and 168 hours. I will be doing this by adding empty zero rows into the data, and then trimming data off the end so that no invalid data is obtained i guess
    # bumping the x data forward by 24 instead of pushing y back by 24 i guess
    print(x_all.shape, y_all.shape)
    zeros_to_append = np.zeros((time_offset, 5))
    x_all = np.concatenate((zeros_to_append, x_all))
    zeros_to_append = np.zeros((time_offset, 1))
    y_all = np.concatenate((y_all, zeros_to_append))
    print(x_all.shape, y_all.shape)

    # slicing out the useless zero data at the end and at the start. The array should now be smaller, but should be offsetted the right amount
    # idk how the nan values are gonna affect this. Hopefully it does not affect it too much
    x_all = x_all[time_offset:-time_offset, :]
    y_all = y_all[time_offset:-time_offset, :]
    print(x_all.shape, y_all.shape)

    x_real = x_all[0:splits, :]
    y_real = y_all[0:splits, :]

    # testing samples
    x_test_real = x_all[splits-timestep:, :]
    y_test_real = y_all[splits-timestep:, :]

    # scaling the samples accordingly
    # most of them fall within a range, and has min and max value, and as such, normalization is choosen i guess
    # Normalization
    scaler_x = MinMaxScaler()
    scaler_x.fit(x_real)
    x_scale = scaler_x.transform(x_real)

    scaler_y = MinMaxScaler()
    scaler_y.fit(y_real)
    y_scale = scaler_y.transform(y_real)

    # setting up the training and validation dataset
    data_generator = TimeseriesGenerator(x_scale, y_scale, length=timestep, sampling_rate=1, batch_size=10000)
    x_timestep, y_timestep = data_generator[0]

    x_train, x_val, y_train, y_val = train_test_split(x_timestep, y_timestep, test_size=0.1, random_state=7567)


    # generating the testing data as well i guess
    x_test_scale = scaler_x.transform(x_test_real)
    y_test_scale = scaler_y.transform(y_test_real)

    data_generator = TimeseriesGenerator(x_test_scale, y_test_scale, length=timestep, sampling_rate=1, batch_size=10000)
    x_test, y_test = data_generator[0]




    # network section starts here
    backend.clear_session()
    init_weight = initializers.GlorotUniform(seed=7567)
    regu_weight = regularizers.l2(0.001)

    backend.clear_session()
    model = Sequential()
    model.add(Input(shape=(timestep,5)))
    model.add(GRU(100, activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_initializer=init_weight, kernel_regularizer=regu_weight))
    model.add(Dense(1, kernel_initializer=init_weight, kernel_regularizer=regu_weight))


    my_optmz = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=my_optmz, loss='mean_squared_error', metrics='mse')
    model_hist = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_val,y_val), verbose=0)




    # predict the train and validation as well i guess
    y_test_pred_scale = model.predict(x_test, verbose=0)
    mse_test_scale = mean_squared_error(y_test, y_test_pred_scale)

    y_test_pred_real = scaler_y.inverse_transform(y_test_pred_scale)
    mse_test_real = mean_squared_error(y_test_real[timestep:,0], y_test_pred_real)


    mse_train_list.append(model_hist.history['mse'][-1])
    mse_val_list.append(model_hist.history['val_mse'][-1])
    # grabbing the scaled mse
    mse_test_list.append(mse_test_scale)

    print(f'timestep of {timestep}')
    print("Mean squared error of the scaled testing dataset:", mse_test_scale)
    print("Mean squared error of the real testing dataset (USD):", mse_test_real)

print(mse_train_list)
print(mse_val_list)
print(mse_test_list)

 # LSTM

In [ ]:
# cleaning out the NaNs
# this will be loading in the regression dataset
# Replace 'your_file.csv' with the path to your CSV file
file_path = 'data/Air_pollution.csv'

# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(file_path)

time_offset_list = [24, 72, 168]
mse_train_list = []
mse_val_list = []
mse_test_list = []

for time_offset in time_offset_list:
    # taking out the nessesary data
    new_df = df[['month', 'day', 'hour', 'pm2.5', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']]

    # dropping out the NaN numbers i guess
    new_df = new_df.dropna()

    # data preprocessing
    # using default values + month, since that was what worked best i guess
    asset_input = pd.DataFrame(new_df, columns=['month', 'DEWP', 'TEMP', 'PRES', 'Iws'])
    x_all = asset_input.values

    asset_output = pd.DataFrame(new_df, columns=['pm2.5'])
    y_all = asset_output.values

    # finding out the length of the dataset, and then getting the first 90% as the data i guess
    splits = int(0.9 * x_all.shape[0])

    timestep = 50

    time_offset = time_offset

    # in this case, i have to offset the training data by 24 hours, 72, and 168 hours. I will be doing this by adding empty zero rows into the data, and then trimming data off the end so that no invalid data is obtained i guess
    # bumping the x data forward by 24 instead of pushing y back by 24 i guess
    print(x_all.shape, y_all.shape)
    zeros_to_append = np.zeros((time_offset, 5))
    x_all = np.concatenate((zeros_to_append, x_all))
    zeros_to_append = np.zeros((time_offset, 1))
    y_all = np.concatenate((y_all, zeros_to_append))
    print(x_all.shape, y_all.shape)

    # slicing out the useless zero data at the end and at the start. The array should now be smaller, but should be offsetted the right amount
    # idk how the nan values are gonna affect this. Hopefully it does not affect it too much
    x_all = x_all[time_offset:-time_offset, :]
    y_all = y_all[time_offset:-time_offset, :]
    print(x_all.shape, y_all.shape)

    x_real = x_all[0:splits, :]
    y_real = y_all[0:splits, :]

    # testing samples
    x_test_real = x_all[splits-timestep:, :]
    y_test_real = y_all[splits-timestep:, :]

    # scaling the samples accordingly
    # most of them fall within a range, and has min and max value, and as such, normalization is choosen i guess
    # Normalization
    scaler_x = MinMaxScaler()
    scaler_x.fit(x_real)
    x_scale = scaler_x.transform(x_real)

    scaler_y = MinMaxScaler()
    scaler_y.fit(y_real)
    y_scale = scaler_y.transform(y_real)

    # setting up the training and validation dataset
    data_generator = TimeseriesGenerator(x_scale, y_scale, length=timestep, sampling_rate=1, batch_size=10000)
    x_timestep, y_timestep = data_generator[0]

    x_train, x_val, y_train, y_val = train_test_split(x_timestep, y_timestep, test_size=0.1, random_state=7567)


    # generating the testing data as well i guess
    x_test_scale = scaler_x.transform(x_test_real)
    y_test_scale = scaler_y.transform(y_test_real)

    data_generator = TimeseriesGenerator(x_test_scale, y_test_scale, length=timestep, sampling_rate=1, batch_size=10000)
    x_test, y_test = data_generator[0]




    # network section starts here
    backend.clear_session()
    init_weight = initializers.GlorotUniform(seed=7567)
    regu_weight = regularizers.l2(0.001)

    model = Sequential()
    model.add(Input(shape=(timestep,5)))
    model.add(LSTM(100, activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_initializer=init_weight, kernel_regularizer=regu_weight))
    model.add(Dense(1, kernel_initializer=init_weight, kernel_regularizer=regu_weight))

    my_optmz = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=my_optmz, loss='mean_squared_error', metrics='mse')
    model_hist = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_val,y_val), verbose=0)




    # predict the train and validation as well i guess
    y_test_pred_scale = model.predict(x_test, verbose=0)
    mse_test_scale = mean_squared_error(y_test, y_test_pred_scale)

    y_test_pred_real = scaler_y.inverse_transform(y_test_pred_scale)
    mse_test_real = mean_squared_error(y_test_real[timestep:,0], y_test_pred_real)


    mse_train_list.append(model_hist.history['mse'][-1])
    mse_val_list.append(model_hist.history['val_mse'][-1])
    # grabbing the scaled mse
    mse_test_list.append(mse_test_scale)

    print(f'timestep of {timestep}')
    print("Mean squared error of the scaled testing dataset:", mse_test_scale)
    print("Mean squared error of the real testing dataset (USD):", mse_test_real)

print(mse_train_list)
print(mse_val_list)
print(mse_test_list)